In [47]:
import tensorflow as tf
import numpy as np
import os

from matplotlib import pyplot as plt 
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import applications
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1
from tensorflow.keras.preprocessing import image_dataset_from_directory

import tensorflow_datasets as tfds 
import copy
import math

from utils.EfficientNet_model import EfficientNetB0_1, EfficientNetB1_1

### 1. Load the imagenet_v2 dataset

In [38]:
#Test set taken from https://github.com/modestyachts/ImageNetV2

_ROOT_URL = 'https://s3-us-west-2.amazonaws.com/imagenetv2public'
_IMAGENET_V2_URLS = {
    'matched-frequency': _ROOT_URL + '/imagenetv2-matched-frequency.tar.gz',
    'threshold-0.7': _ROOT_URL + '/imagenetv2-threshold0.7.tar.gz',
    'topimages': _ROOT_URL + '/imagenetv2-topimages.tar.gz',
}
fname = "imagenet_v2_matched_frequency"
local_file_path= tf.keras.utils.get_file(
    fname, _IMAGENET_V2_URLS['matched-frequency'], 
    cache_subdir='datasets', hash_algorithm='auto',
    extract=True)


local_dir_path = os.path.dirname(local_file_path)
data_dir = os.path.join(local_dir_path, 'imagenet-matched-frequency-format-val')
print(data_dir)


/Users/valentinedhauteville/.keras/datasets/imagenet-matched-frequency-format-val


In [56]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

validation_dataset = image_dataset_from_directory(data_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  label_mode = "categorical",
                                                  image_size=IMG_SIZE)

Found 20000 files belonging to 1000 classes.


In [67]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

image_batch, label_batch = next(iter(validation_dataset))
image_batch.shape

TensorShape([32, 224, 224, 3])

### 2. EfficientNet own versus EfficientNet Keras

We load imagenet weights (taken off the internet) into the model we created and compare the performance with:
* The keras implementation loaded with imagenet weights (that is instantiated with the weights = "imagenet" parameter. We use this as the reference. In the best of worlds, this should give us a performance similar to the official paper's ( ~77.1% top-1 accuracy)
* The keras implementation loaded with imagenet weights we used for our model. This should give us the same performance as our model's, otherwise this means something is off with our implementation

### a. Load the models

In [ ]:
#Number of layers

model0_1 = EfficientNetB0_1()
print(f"num_layers in self implemented: {len(model0_1.layers)}")

In [ ]:
#Load the keras model as reference

#with imagenet weights
model0 = EfficientNetB0(weights="imagenet", include_top=True)

#no pretrained weights
model00 = EfficientNetB0(weights= None, include_top=True)
print(f"num_layers total keras: {len(model00.layers)}" )


In [13]:
#Number of layers with parameters/weights

layers = model0_1.layers
filtered_layers = []
for layer in layers:
    weights = layer.weights
    if weights:
        filtered_layers.append(layer)
print(f"num_layers trainable in self implemented EfficientNet_B0: {len(filtered_layers)}" )        

layers = model00.layers
filtered_layers = []
for layer in layers:
    weights = layer.weights
    if weights:
        filtered_layers.append(layer)

print(f"num_layers trainable in keras EfficientNet-B0: {len(filtered_layers)}" )    

num_layers trainable in self implemented EfficientNet_B0: 132
num_layers trainable in keras EfficientNet-B0: 132


In [ ]:
#model0_1.summary()


In [ ]:
#model00.summary()


### b. Get the weights

We tried three different optional repositories for the weights.

In [27]:
#Number of layers with parameters/weights

layers = model0_1.layers
filtered_layers = []
for layer in layers:
    weights = layer.weights
    if weights:
        filtered_layers.append(layer)
print(f"num_layers trainable in self implemented EfficientNet_B0: {len(filtered_layers)}" )        

layers = model00.layers
filtered_layers = []
for layer in layers:
    weights = layer.weights
    if weights:
        filtered_layers.append(layer)

print(f"num_layers trainable in keras EfficientNet-B0: {len(filtered_layers)}" )        

num_layers trainable in self implemented EfficientNet_B0: 132
num_layers trainable in keras EfficientNet-B0: 132


In [65]:
# Method 1: From the official TPU repo 
#https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet
# utils/efficientnet_weight_update_util.py is a script copied from https://github.com/tensorflow/
#tensorflow/blob/master/tensorflow/python/keras/applications/efficientnet_weight_update_util.py
# to convert a ckpt file to h5 file
def download_weights_from_tpu_repo(model_name):
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/{model_name}.tar.gz -O {model_name}.tar.gz
    !tar xf {model_name}.tar.gz

    !wget https://raw.githubusercontent.com/tensorflow/tensorflow\
        /master/tensorflow/python/keras/applications/efficientnet_weight_update_util.py
    # convert to H5
    !python utils/efficientnet_weight_update_util.py --model b0 --ckpt \
            {model_name}/model.ckpt --o {model_name}.h5

    weights_path = f"{model_name}.h5"
    return weights_path

#Method 2, using checkpoints published on https://github.com/qubvel/efficientnet/
def download_weights_from_individual_contrib(model_name):
    !wget https://github.com/qubvel/efficientnet/releases/download/v0.0.1/{model_name}_imagenet_1000.h5
    weights_path = f"model_name_imagenet_1000.h5"
    return weights_path

In [28]:
#Method 3: from the Keras source code. The hash below corresponds to EfficientNetB0 weights

# file_suff = '_weights_tf_dim_ordering_tf_kernels_autoaugment.h5'
# file_hash = 'e9e877068bd0af75e0a36691e03c072c' #file hash for B0
# model_name = "efficientnet-b0"
# file_name =  model_name + file_suff
# BASE_WEIGHTS_PATH = (
#     'https://github.com/Callidior/keras-applications/'
#     'releases/download/efficientnet/')

# weights_path = tf.keras.utils.get_file(file_name, 
#                                        BASE_WEIGHTS_PATH + file_name,
#                                        cache_subdir='models', file_hash=file_hash)

# model_1.load_weights(weights_path, by_name=True)
# model_00.load_weights(weights_path, by_name=True)


## 4.  Evaluate models

In [18]:
import functools
top5_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'


#### EfficientNet-B0

In [61]:
#Keras model with pretrained ImageNet weights
model0.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy', top5_acc])

loss, accuracy, top5_acc = model0.evaluate(validation_dataset)
print('Test accuracy :', accuracy)

625/625 [==============================] - 1347s 2s/step - loss: 1.8168 - accuracy: 0.5971 - top5_acc: 0.8214
Test accuracy : 0.597100019454956


In [57]:
weights_path = download_weights_from_tpu_repo("efficientnet-b0")


--2020-12-19 23:23:02--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckpts/efficientnet-b0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:4007:80c::2010, 2a00:1450:4007:810::2010, 2a00:1450:4007:812::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:4007:80c::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47390720 (45M) [application/gzip]
Saving to: ‘efficientnet-b0.tar.gz’

efficientnet-b0.tar 100%[===================>]  45.20M   525KB/s    in 2m 19s  

2020-12-19 23:25:24 (332 KB/s) - ‘efficientnet-b0.tar.gz’ saved [47390720/47390720]

--2020-12-19 23:25:24--  https://raw.githubusercontent.com/tensorflow/tensorflow
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.120.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.120.133|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2020-12-19 23:25

In [58]:
#Keras model with our downloaded ImageNet weights

model00.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy', top5_acc])

model00.load_weights(weights_path, by_name=True)

loss, accuracy, top_5 = model00.evaluate(validation_dataset)
print('Test accuracy :', accuracy)

625/625 [==============================] - 1247s 2s/step - loss: 3.0190 - accuracy: 0.3994 - top5_acc: 0.6435
Test accuracy : 0.399399995803833


In [59]:
model0_1.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics=['accuracy', top5_acc])

model0_1.load_weights(weights_path,  by_name=True)

loss, accuracy, top_5 = model0_1.evaluate(validation_dataset)
print('Test accuracy :', accuracy)

625/625 [==============================] - 1207s 2s/step - loss: 3.0659 - accuracy: 0.3963 - top5_acc: 0.6353
Test accuracy : 0.39629998803138733
